# Recomendador de peliculas

In [1]:
import pandas as pd
import numpy as np
import sqlite3



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
url_movies = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'

url_credits = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

In [3]:
df_movies = pd.read_csv(url_movies)

df_credits = pd.read_csv(url_credits)

df_movies.to_csv('../data/raw/movies.csv')
df_credits.to_csv('../data/raw/credits.csv')

In [4]:
df_movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2648,15000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.dfilmscorp.ca/,75033,"[{""id"": 5565, ""name"": ""biography""}, {""id"": 585...",en,Winnie Mandela,A drama that chronicles the life of Winnie Man...,1.897761,[],[],2011-09-16,0,107.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Her courage inspired a nation.,Winnie Mandela,5.2,5
3214,9000000,"[{""id"": 878, ""name"": ""Science Fiction""}]",NaN,8069,"[{""id"": 9835, ""name"": ""sexual fantasy""}, {""id""...",en,Barbarella,"In the far future, a highly sexual woman is ta...",14.171759,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1968-10-10,2500000,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,See Barbarella do her thing!,Barbarella,5.7,146
3053,6000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,87818,"[{""id"": 9253, ""name"": ""slapstick""}, {""id"": 159...",en,Movie 43,The film is composed of multiple comedy shorts...,35.350303,"[{""name"": ""Greenestreet Films"", ""id"": 2150}, {...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-01-01,32438988,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The biggest cast ever assembled for the most o...,Movie 43,4.4,797
3941,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",http://www.letterstogodthemovie.com,35691,"[{""id"": 3344, ""name"": ""letter""}, {""id"": 3929, ...",en,Letters to God,A young boy fighting cancer writes letters to ...,3.644872,"[{""name"": ""Mercy Creek Entretainment"", ""id"": 5...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-04-09,0,110.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Letters to God,6.5,37
1387,34000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",NaN,1883,"[{""id"": 187, ""name"": ""islam""}, {""id"": 242, ""na...",en,Malcolm X,The biopic of the controversial and influentia...,19.822791,"[{""name"": ""JVC Entertainment"", ""id"": 182}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1992-11-18,48169908,202.0,"[{""iso_639_1"": ""ar"", ""name"": ""\u0627\u0644\u06...",Released,NaN,Malcolm X,7.2,370


In [5]:
df_credits.sample(5)

,movie_id,title,cast,crew
823,8963,Crimson Tide,"[{""cast_id"": 3, ""character"": ""Lt. Commander Ro...","[{""credit_id"": ""566b4f729251415ec5005aac"", ""de..."
4642,1424,November,"[{""cast_id"": 1, ""character"": ""Sophie Jacobs"", ...","[{""credit_id"": ""52fe42f6c3a36847f803001f"", ""de..."
3826,352978,Chain of Command,"[{""cast_id"": 0, ""character"": ""James Webster"", ...","[{""credit_id"": ""55c08500c3a36823c9001441"", ""de..."
2353,11457,Life as a House,"[{""cast_id"": 1, ""character"": ""George Monroe"", ...","[{""credit_id"": ""52fe44449251416c7502ea11"", ""de..."
2298,13523,Sex Drive,"[{""cast_id"": 1, ""character"": ""Ian"", ""credit_id...","[{""credit_id"": ""53745e9dc3a3681518002464"", ""de..."


Voy a comprobar si mis titulos son unicos, las instrucciones indican que deberia hacer un join a traves de la columna titulo, pero si hay algun titulo repetido, esto borraria y solo dejaria uno.

In [6]:
df_movies.shape

(4803, 20)

In [7]:
df_movies['title'].nunique()

4800

In [8]:
repeated_titles = df_credits['title'][df_credits['title'].duplicated()].unique()
print(repeated_titles)

['The Host' 'Out of the Blue' 'Batman']


Veo que si que tengo duplicados, voy a comprobar las filas a ver si son duplicados completos

In [9]:
resultado = df_movies[df_movies['title'] == 'Batman']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1359,35000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",NaN,268,"[{""id"": 848, ""name"": ""double life""}, {""id"": 84...",en,Batman,The Dark Knight of Gotham City begins his war ...,44.104469,"[{""name"": ""PolyGram Filmed Entertainment"", ""id...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",1989-06-23,411348924,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Have you ever danced with the devil in the pal...,Batman,7.0,2096
4267,1377800,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...",NaN,2661,"[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...",en,Batman,The Dynamic Duo faces four super-villains who ...,9.815394,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1966-07-30,0,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's Here Big As Life In A Real Bat-Epic,Batman,6.1,203


In [10]:
resultado = df_movies[df_movies['title'] == 'Out of the Blue']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3647,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,39269,"[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...",en,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,0.679351,"[{""name"": ""Robson Street"", ""id"": 71953}]","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",1980-05-01,0,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A harrowing drama from the director of Easy Rider,Out of the Blue,6.5,17
3693,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,10844,"[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...",en,Out of the Blue,Ordinary people find extraordinary courage in ...,0.706355,[],"[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}]",2006-10-12,0,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The true story of a small town massacre,Out of the Blue,5.9,18


In [11]:
resultado = df_movies[df_movies['title'] == 'The Host']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
972,44000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,72710,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Host,A parasitic alien soul is injected into the bo...,42.933027,"[{""name"": ""Nick Wechsler Productions"", ""id"": 8...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-03-22,63327201,125.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You will be one of us,The Host,6.0,1817
2877,11000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...",http://www.hostmovie.com/,1255,"[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...",ko,괴물,Gang-du is a dim-witted man working at his fat...,27.655270,"[{""name"": ""Cineclick Asia"", ""id"": 685}, {""name...","[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",2006-07-27,88489643,119.0,"[{""iso_639_1"": ""ko"", ""name"": ""\ud55c\uad6d\uc5...",Released,Monsters are real.,The Host,6.7,537


Puedo observar que son completamente diferentes, por lo tanto unir por titulo no seria factible, ya que estos repetidos se eliminarian

Voy a comprobar duplicados en la columna ID, en caso de que no los haya sera la que utilice para unir mis dos datasets

In [12]:
print(df_movies['id'].nunique())
print(df_credits['movie_id'].nunique())

4803
4803


Puedo observar que no hay duplicados en los ids de cada uno de mis datasets, intentare hacer el join a traves de ellos, en caso de obtener mas filas de las que tengo originalmente, sera porque no coinciden entre si

In [13]:
# Creo mi database y la conexion a ella
conn = sqlite3.connect('../data/processed/movie_database.db')

# Creo el objeto cursor que interactuara con ella
cursor = conn.cursor()

# Creo mis tablas
df_movies.to_sql('tabla_movies', conn, if_exists='replace', index=False)
df_credits.to_sql('tabla_credits', conn, if_exists='replace', index=False)

4803

In [14]:
query = """
CREATE TABLE IF NOT EXISTS tabla_resultados AS 
SELECT tabla_credits.movie_id, tabla_movies.title, tabla_movies.overview, 
       tabla_movies.genres, tabla_movies.keywords, tabla_credits.cast, tabla_credits.crew
FROM tabla_movies
INNER JOIN tabla_credits
ON tabla_movies.id = tabla_credits.movie_id;
"""


cursor.execute(query)
conn.commit()

In [15]:
df = pd.read_sql('select * from tabla_resultados', conn)

conn.close()

In [16]:
df.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1242,1683,The Reaping,"Katherine Morrissey, a former Christian missio...","[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 1261, ""name"": ""river""}, {""id"": 2618, ""...","[{""cast_id"": 1, ""character"": ""Katherine"", ""cre...","[{""credit_id"": ""52fe430bc3a36847f8036677"", ""de..."
623,38778,Dinner for Schmucks,Rising executive Tim Wagner works for a boss w...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 2161, ""name"": ""mouse""}, {""id"": 2415, ""...","[{""cast_id"": 4, ""character"": ""Barry"", ""credit_...","[{""credit_id"": ""52fe46df9251416c910628d5"", ""de..."
1707,924,Dawn of the Dead,A group of surviving people take refuge in a s...,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 27, ""na...","[{""id"": 972, ""name"": ""refugee""}, {""id"": 1583, ...","[{""cast_id"": 22, ""character"": ""Ana"", ""credit_i...","[{""credit_id"": ""52fe428ec3a36847f8027c57"", ""de..."
4123,20653,Roadside Romeo,This is the story of Romeo. A dude who was liv...,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",[],"[{""cast_id"": 1, ""character"": ""Romeo"", ""credit_...",[]
3589,284293,Still Alice,"Alice Howland, happily married with three grow...","[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 456, ""name"": ""mother""}, {""id"": 1992, ""...","[{""cast_id"": 1, ""character"": ""Dr. Alice Howlan...","[{""credit_id"": ""54669f3c22136e7bec0000b3"", ""de..."


In [17]:
df.shape

(4803, 7)

Con esto he conseguido unir las dos bases de datos sin eliminar las filas repetidas.

nearest neighbours, tf-idf

In [18]:
import json

df = df.assign(genres=lambda x: x['genres'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(keywords=lambda x: x['keywords'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(cast=lambda x: x['cast'].apply(lambda y: [item['name'] for item in json.loads(y)][:3] if pd.notna(y) else None))

# columna crew
df = df.assign(crew=lambda x: x['crew'].apply(lambda y: next((item['name'] for item in json.loads(y) if item.get('job') == 'Director'), None) if pd.notna(y) else None))

df = df.assign(overview=lambda x: x['overview'].apply(lambda y: [str(y)] if pd.notna(y) else []))

In [19]:
df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [20]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [21]:
from funciones import remove_spaces

# Borro los espacios en las columnas
df['genres'] = df['genres'].apply(remove_spaces)
df['cast'] = df['cast'].apply(remove_spaces)
df['crew'] = df['crew'].apply(remove_spaces)
df['keywords'] = df['keywords'].apply(remove_spaces)

In [22]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",SamMendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",ChristopherNolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",AndrewStanton


In [23]:
df["genres"] = df["genres"].apply(lambda x: [str(genre) for genre in x])
df["keywords"] = df["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df["cast"] = df["cast"].apply(lambda x: [str(actor) for actor in x])

# Convierto mi columna 'crew' en una lista para poder hacer el .join
df["crew"] = df["crew"].apply(lambda x: [x] if pd.notna(x) and isinstance(x, str) else [])

In [24]:
df.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [25]:
df["tags"] = df["overview"] + df["genres"] + df["keywords"] + df["cast"] + df["crew"]
df["tags"] = df["tags"].apply(lambda x: ",".join(x).replace(",", " "))

In [26]:
df.drop(columns=["genres", "keywords", "cast", "crew", "overview"], inplace=True)

df.iloc[0].tags

'In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'